# Project: Solar Power Generation Data

# Introduction
This data has been gathered at two solar power plants in India over a 34 day period. It has two pairs of files - each pair has one power generation dataset and one sensor readings dataset. The power generation datasets are gathered at the inverter level - each inverter has multiple lines of solar panels attached to it. The sensor data is gathered at a plant level - single array of sensors optimally placed at the plant.

## Contributions

## Extract the data

we assume data files are in the same directory as the notebook.
The data can be accessed at <https://www.kaggle.com/anikannal/solar-power-generation-data/download>

In [ ]:
plant_1_gen = pd.read_csv("Plant_1_Generation_data.csv")
plant_2_gen = pd.read_csv("Plant_2_Generation_data.csv")
plant_1_wea = pd.read_csv("Plant_1_Weather_Sensor_data.csv")
plant_2_wea = pd.read_csv("Plant_2_Weather_Sensor_data.csv")

## Check columns for missing values


### Plant 1

In [ ]:
plant1_wea = plant_1_wea.isnull().sum()
print(plant1_wea)

Both weather and generation data has no null values, however this is time series data so what if there are dicontinuity in the time?

In [ ]:
temp_gen1 = plant_1_gen.copy()
temp_gen1.head()

### Plant 2

In [ ]:
plant2_info = plant_2_gen.isnull().sum()
print(plant2_info)

In [ ]:
plant2_wea = plant_2_wea.isnull().sum()
print(plant2_wea)

In [ ]:
plant1_info = plant_1_gen.isnull().sum()
print(plant1_info)

## How many sensors are at each plant?

In [ ]:
print("Plant 1 has",plant_1_gen['SOURCE_KEY'].nunique(),"sensors")
print("Plant 2 has",plant_2_gen['SOURCE_KEY'].nunique(),"sensors")

In [ ]:
#Data for 34 days and continuous dates
numb_daws = pd.to_datetime(plant_1_gen['DATE_TIME']).dt.date.unique()

plant_1_gen['DATE_TIME'] = pd.to_datetime(plant_1_gen['DATE_TIME'])

data = plant_1_gen.groupby(['SOURCE_KEY']).apply(lambda x: x.sort_values(by=['DATE_TIME'], ascending=True))

time_gaps = pd.DataFrame(data.DATE_TIME.iloc[1:].diff())
non_continuous_dates = time_gaps.loc[(time_gaps.DATE_TIME.dt.seconds > 15*60) | (time_gaps.DATE_TIME.dt.seconds < -15*60)]
non_continuous_dates.index.set_names(["SOURCE_KEY", "Index"], inplace=True)

# non_continuous_dates.index.get_level_values(1)

# plot the non-continuous dates to visualize time gaps. Use the multiindex to identify the sensors and the index of the time gaps

# fill NaT values with 15 min timedelta, in non_continuous_dates
non_continuous_dates.fillna(pd.Timedelta(15*60, unit='s'), inplace=True)

time_gaps.plot(x='DATE_TIME', y='DATE_TIME', kind='scatter')
non_continuous_dates.plot(x='DATE_TIME', y='DATE_TIME', kind='scatter')


#### Some inverters have more data points then the others

In [ ]:
#Inverterids
print("Plant 1 statistics \n", plant_1_gen.SOURCE_KEY.value_counts())
print("Plant 2 statistics \n", plant_2_gen.SOURCE_KEY.value_counts())

## Data visualization

### **Convert datetime column to datetime format**

In [ ]:
#Plant 1 weather data
plant_1_wea['DATE_TIME'] = pd.to_datetime(plant_1_wea['DATE_TIME']) 
plant_1_wea['TIME'] = plant_1_wea['DATE_TIME'].dt.time 
#convert datetime column to just date
plant_1_wea['DATE'] = pd.to_datetime(plant_1_wea['DATE_TIME'].dt.date)
print(plant_1_wea['DATE'])

#Plant 2 weather data
plant_2_wea['DATE_TIME'] = pd.to_datetime(plant_2_wea['DATE_TIME']) 
plant_2_wea['TIME'] = plant_2_wea['DATE_TIME'].dt.time 
#convert datetime column to just date
plant_2_wea['DATE'] = pd.to_datetime(plant_2_wea['DATE_TIME'].dt.date)


### **Analysis of weather dataset for both the plants**

In [ ]:
# Comparing both plants
# Daily Irradiation
ambient_compare = sns.lineplot(x='DATE', y='IRRADIATION', data=plant_1_wea, err_style='band', label='Plant 1')
sns.lineplot(x='DATE', y='IRRADIATION', data=plant_2_wea, err_style='band', label='Plant 2', ax=ambient_compare)
plt.ylabel('Irradiation')
plt.xlabel('Date')
plt.title('Daily Solar Irradiation for Both Plants')
plt.xticks(rotation=45)
plt.show()

# The mean of solar Irradiation for both plants are similar
mean_irradiationplant1 = plant_1_wea['IRRADIATION'].mean()
print('Mean of solar irradition from Plant1', mean_irradiationplant1)
mean_irradiationplant2 =  plant_2_wea['IRRADIATION'].mean()
print('Mean of solar irradition from Plant2', mean_irradiationplant2)

In [ ]:
# Daily Module Temperature
modtemp_compare = sns.lineplot(x='DATE', y='MODULE_TEMPERATURE', data=plant_1_wea, err_style='band', label='Plant 1')
sns.lineplot(x='DATE', y='MODULE_TEMPERATURE', data=plant_2_wea, err_style='band', label='Plant 2', ax=modtemp_compare)
plt.ylabel('Module Temperature')
plt.xlabel('Date')
plt.title('Daily Module Temperature for Both Plants')
plt.xticks(rotation=45)
plt.show()

# The mean of Module Temperature for both plants (Plant 1 is lower then Plant 2)
mean_moduletempplant1 = plant_1_wea['MODULE_TEMPERATURE'].mean()
print('Mean of Module Temperature from Plant1', mean_moduletempplant1)
mean_moduletempplant2 =  plant_2_wea['MODULE_TEMPERATURE'].mean()
print('Mean of Module Temperature from Plant2', mean_moduletempplant2)


In [ ]:
ambtemp_compare = sns.lineplot(x='DATE', y='AMBIENT_TEMPERATURE', data=plant_1_wea, err_style='band', label='Plant 1')
sns.lineplot(x='DATE', y='AMBIENT_TEMPERATURE', data=plant_2_wea, err_style='band', label='Plant 2', ax=ambtemp_compare)
plt.ylabel('Ambient Temperature')
plt.xlabel('Date')
plt.title('Daily Ambient Temperature for Both Plants')
plt.xticks(rotation=45)
plt.show()


# The mean of Ambient Temperature for both plants (Plant 1 is lower then Plant 2)
mean_ambienttempplant1 = plant_1_wea['AMBIENT_TEMPERATURE'].mean()
print('Mean of Ambient Temperature from Plant1', mean_ambienttempplant1)
mean_ambienttempplant2 =  plant_2_wea['AMBIENT_TEMPERATURE'].mean()
print('Mean of Ambient Temperature from Plant2', mean_ambienttempplant2)

#### Observations

1. The mean solar irradiation values for both plants are similar.
2. The mean module temperature of Plant 1 is  lower than Plant 2 most of the time.
3. The mean ambient temperature of Plant 1 is much lower than Plant 2.


## Data cleaning


#### Transform and merge the datasets

##### Drop unwanted columns.

In [ ]:
df_weather1 = plant_1_wea.drop(['PLANT_ID', 'SOURCE_KEY'], axis=1)
df_plant1 = plant_1_gen.drop(['PLANT_ID'], axis=1)
df_weather2 = plant_2_wea.drop(['PLANT_ID', 'SOURCE_KEY'], axis=1)
df_plant2 = plant_2_gen.drop(['PLANT_ID'], axis=1)

In [ ]:
df_plant1["Human_key"] = df_plant1["SOURCE_KEY"].map({val: f"S{i:02d}" for i, val in enumerate(df_plant1["SOURCE_KEY"].unique())})
df_plant1.sample(20)

In [ ]:
#Formatedattime
df_plant1['DATE_TIME']= pd.to_datetime(df_plant1['DATE_TIME'],format='%d-%m-%Y %H:%M')
df_weather1['DATE_TIME']= pd.to_datetime(df_weather1['DATE_TIME'],format='%Y-%m-%d %H:%M:%S')
#df_plant1.head()
df_plant2['DATE_TIME']= pd.to_datetime(df_plant2['DATE_TIME'],format='%Y-%m-%d %H:%M:%S')
df_weather2['DATE_TIME']= pd.to_datetime(df_weather2['DATE_TIME'],format='%Y-%m-%d %H:%M:%S')

In [ ]:
df_plant_weather1 = df_plant1.merge(df_weather1, left_on='DATE_TIME', right_on='DATE_TIME')
df_plant_weather2 = df_plant2.merge(df_weather2, left_on='DATE_TIME', right_on='DATE_TIME')

print(df_plant_weather1.head())
print(df_plant_weather2.head())

### Imputation

## Data Analysis

### **Data Correlation Analysis**


Observations:
1. High correlation between DC Power and AC power generation
2. High correlation between DC Power and IRRADIATION
3. Strong correlation between DC Power, AC Power and Module Temperature and Ambient Temperature

In [ ]:
corr_matrix = df_plant_weather1.corr()

corr_matrix["DC_POWER"].sort_values(ascending=False)

In [ ]:
attributes = ["DC_POWER", "AC_POWER", "IRRADIATION",
              "MODULE_TEMPERATURE","AMBIENT_TEMPERATURE"]

plt.subplot(1, 2, 1)
scatter_plot = scatter_matrix(df_plant_weather1[attributes], figsize=(15, 15))

plt.subplot(1, 2, 2)
fig_corr = sns.heatmap(corr_matrix,cmap="YlGnBu", annot=True) # TODO - subplot + margin

**Analysis of DC power generated from each Source Keys.**
1. TheDistribution DC power generation plot shows multiple occasion where power generated was zero during daytime.
2.  plot of solar irradiation exhibits that the solar radiation never dropped to a lower value at day time. 
3. Analysis shows some inverters received no DC power even through there was enough sunlight
4. It could be concluded that the DC power generated and solar irradiation has a linear relationship.

### **Stacked Visualization of Power Generation**

In [ ]:
#DC power generated from each source keys
sources=df_plant_weather1.copy()
sources['time']=sources['DATE_TIME'].dt.time
sources.set_index('time').groupby('SOURCE_KEY')['DC_POWER'].plot(style='o',legend=True,figsize=(20,10))
plt.title('DC Power generated for all interverters(source_keys)',size=17)
plt.ylabel('DC POWER ( kW )',color='purple',fontsize=17)
plt.show()

In [ ]:
#Inverters with lower performace then rest "1BY6WEcLGh8j5v7", "bvBOhCH3iADSZry"
SOURCE = df_plant_weather1.groupby('SOURCE_KEY').agg({'DC_POWER': ['mean', 'min', 'max','median']})['DC_POWER']
SOURCE["SOURCE_KEY"] = SOURCE.index
mean = SOURCE['mean'].mean()
SOURCE["mean_new"] = SOURCE['mean'] - mean
print(SOURCE.head())

SOURCE.plot(x="SOURCE_KEY", y="mean_new")

## Research

Solar power plants generate a vast amount of data which can be used to determine how a plant is performing and what is impacting its performance. Taking this a step further, a plant’s performance can be optimized by leveraging advanced analytics. Advanced analytics includes formulating statistical matrices, implementing actionable performance alarms, and conducting artificial intelligent models to predict future performance. A relatively accurate energy prediction can be calculated by incorporating weather forecasts, a plant’s historical performance
This method requires substantial computing power and considerable time to develop accurate predictions. To illustrate this, this project will attempt to analyze power generation data in tandem with weather data to predict near-future (days) performance with data gathered from two solar power plants in India over a 34-day period.


### Select features of interest / Problem Statement

Prediction of power generation using different ML techniques.

### Choose an appropriate ML model based on the dataset and your problem statement.

# ML Techniques


### Scoring Function

In [ ]:
def scoring(y_test, y_pred):
    R2_train = r2_score(y_test, y_pred)
    mse_train = mean_squared_error(y1_train, Y1_pred_train)
    rmse_train = np.sqrt(mse_train)
    return(R2_train, mse_train, rmse_train)

Linear Regression
From the data analysis and corerelation analsysis it was inferred that there is a linear relation between the DC power generated andthe solar irradiation. This relation can be modeled using a simple linear relationship as below,
𝑃(𝑡) = 𝑎 + 𝑏 . 𝐸(𝑡) where p(t) denotes the power generated and the E(t) denotes the solar irradiation. So we will run a linear regression model on our datasets.

### Model 1: Linear Regression for Plant 1 dataset

In [ ]:
X1=df_plant_weather1[['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE']] # Features 
y1=df_plant_weather1['DC_POWER'] # independent var / predictor

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.20, random_state=1)

print("Shape of each Dataset : ")
print("X Train Shape = ",X1_train.shape)
print("Y Train Shape = ",y1_train.shape)
print("X Test Shape  = ",X1_test.shape)
print("Y Test Shape  = ",y1_test.shape)

In [ ]:
lm = LinearRegression()

Fit on training data

In [ ]:
lm.fit(X1_train, y1_train)
lm.intercept_, lm.coef_

In [ ]:
print ('Plant 1 linear regression cofficients are:', X1.columns, lm.coef_ )
print('Plant 1 intercept', lm.intercept_)

Prediction from model

In [ ]:
Prediction_plant1 = lm.predict(X1_test)
Prediction_plant1
Y1_pred_train = lm.predict(X1_train)
Y1_pred_test = lm.predict(X1_test)

print ('Prediction Train dataset', Y1_pred_train)
print ('Prediction Train dataset', Y1_pred_test)

Actual Solar Output Values vs Predicted Values for Plant 1 using Linear Regression (This also includes faulty days dataset)

In [ ]:
plt.scatter(y1_test, Y1_pred_test,color="b", label="Predicted Output")
plt.legend()
plt.title('Actual Solar Output Values vs Predicted Values for Plant 1')
plt.xlabel('Predicted Output')
plt.ylabel('Actual Output')

Model Evaluation for Linear Regression Model 1 using RMSE( Plant1 dataset)

In [ ]:
print("Model Evaluation for Linear Regression Model 1 using RMSE( Plant1)")
print("")
RMSE_train_1 = np.sqrt( metrics.mean_squared_error(y1_train, Y1_pred_train))
RMSE_test_1 = np.sqrt(metrics.mean_squared_error(y1_test, Y1_pred_test))
MAE1 = metrics.mean_absolute_error(y1_test, Y1_pred_test)
MSE1 = metrics.mean_squared_error(y1_test, Y1_pred_test)
RMSE1 = np.sqrt(metrics.mean_squared_error(y1_test, Y1_pred_test))
print('RMSE for training set = {}'.format(round(RMSE_train_1,2)))
print('RMSE for test set = {}'.format(round(RMSE_test_1,2)))
print('MAE: ', MAE1)
print('MSE: ',MSE1)

### Model 1 : Linear Regression for Plant 2 dataset

In [ ]:
X12=df_plant_weather2[['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE','IRRADIATION']] # Features 
y12=df_plant_weather2['DC_POWER'] # independent var / predictor

X12_train, X12_test, y12_train, y12_test = train_test_split(X12, y12, test_size=0.20, random_state=1)

print("Shape of each Dataset : ")
print("X Train Shape = ",X12_train.shape)
print("Y Train Shape = ",y12_train.shape)
print("X Test Shape  = ",X12_test.shape)
print("Y Test Shape  = ",y12_test.shape)

In [ ]:
lm = LinearRegression()

Fit on training data for Plant 2

In [ ]:

lm.fit(X12_train, y12_train)
lm.intercept_, lm.coef_

Prediction from model

In [ ]:
Prediction_plant2 = lm.predict(X12_test)
Prediction_plant2
Y12_pred_train = lm.predict(X12_train)
Y12_pred_test = lm.predict(X12_test)



print ('Prediction Train dataset', Y12_pred_train)
print ('Prediction Train dataset', Y12_pred_test)

scoring(y12_test, Prediction_plant2)

Actual Solar Output Values vs Predicted Values for Plant 2 using Linear Regression (This also includes faulty days dataset)

In [ ]:
plt.scatter(y12_test, Y12_pred_test, color="r", label="Predicted Output for test")
plt.scatter(y12_train, Y12_pred_train, color="b", label="Predicted Output for train")
plt.legend()
plt.title('Actual Solar Output Values vs Predicted Values for Plant 2')
plt.xlabel('Predicted Output')
plt.ylabel('Actual Output')

Accuracy score for Plant 2 using Linear Regression

In [ ]:
train_score_12 = lm.score(X12_train, y12_train)
test_score_12 = lm.score(X12_test, y12_test)

print("Model 1 Plant 2 accuracy score: ")
print("")
print("Train Score = ",round(train_score_12*100,0),"%")
print("Test Score  = ",round(test_score_12*100,0), "%")

### Model 2: Random Forest for Plant2 dataset 

Select columns that will be used to create train and test dataset

In [ ]:
model_2 = df_plant_weather2
model_2.head()

In [ ]:
X2 = model_2[['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']] # Features
y2 = model_2['DC_POWER'] # Target

X2_train, X2_test, y2_train, y2_test=train_test_split(X2, y2, test_size=0.20, random_state=1, shuffle=False) 

print("Shape of each New Dataset : ")
print("X Train Shape = ",X2_train.shape)
print("Y Train Shape = ",y2_train.shape)
print("X Test Shape  = ",X2_test.shape)
print("Y Test Shape  = ",y2_test.shape)

In [ ]:

RF = RandomForestRegressor()
RF.fit(X2_train, y2_train)

In [ ]:
y2_pred_train = RF.predict(X2_train)
y2_pred_test = RF.predict(X2_test)

Model Evaluation for Random Forest

In [ ]:
print("Model Evaluation for Random Forest Model 2 using RMSE")

RMSE_train_2 = np.sqrt( metrics.mean_squared_error(y2_train, y2_pred_train))
RMSE_test_2 = np.sqrt(metrics.mean_squared_error(y2_test, y2_pred_test))
MSE_train_2 = ( metrics.mean_squared_error(y2_train, y2_pred_train))
print('RMSE for training set is {}'.format(round(RMSE_train_2,2)))
print('RMSE for test set is {}'.format(round(RMSE_test_2,2)))
print('MSE for train set is {}'.format(round(MSE_train_2,2)))

Accuracy score for Model 2

In [ ]:
train_score_2 = RF.score(X2_train, y2_train)
test_score_2 = RF.score(X2_test, y2_test)

print("Model 2 accuracy score: ")
print("")
print("Train Score = ",round(train_score_2*100,0),"%")
print("Test Score  = ",round(test_score_2*100,0), "%")

Actual Solar Output Values vs Predicted Values for Plant 2 using Random Forest Regresor (This also includes faulty days dataset)

In [ ]:
# This also includes faulty days dataset
plt.scatter(y2_test, y2_pred_test,color="r", label="Predicted Output test")
plt.scatter(y2_train, y2_pred_train,color="b", label="Predicted Output train")
#plt.scatter(Y_test, DC_POWER, label="Measured Output")
#plt.plot(X, Y, "b.")
plt.legend()
plt.title('Actual Solar Output Values vs Predicted Values for Plant 1 using Random Forest')
plt.xlabel('Predicted Output')
plt.ylabel('Actual Output')

Feature Scaling

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X2_train)
X_test = sc.transform(X2_test)

Running the model after Feature scaling

In [ ]:
regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X2_train, y2_train)
y3_pred = regressor.predict(X2_test)


Accuracy scorefor Model 2 after scaling data

In [ ]:
train_score_2 = RF.score(X2_train, y2_train)
test_score_2 = RF.score(X2_test, y2_test)

print("Model 2 accuracy score: ")
print("")
print("Train Score = ",round(train_score_2*100,0),"%")
print("Test Score  = ",round(test_score_2*100,0), "%")

### Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")

scores = cross_val_score(lm, X12, y12, cv=10)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


scores2 = cross_val_score(svm_reg, X12, y12, cv=10)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores2.mean(), scores2.std()))

In [ ]:
scores = cross_val_score(lm, X1, y1, cv=10)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

In [ ]:

scores = cross_val_score(RF, X2_train, y2_train, cv=10)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

In [ ]:
# import timeseriessplit
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

def Cross_Validation(model, X, y, n_splits=10):
    tscv = TimeSeriesSplit(n_splits=n_splits)

    scores = []
    for train_index, test_index in tscv.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        Y_train, Y_test = y.iloc[train_index], y.iloc[test_index] 

        model.fit(X_train, Y_train)
        predict = model.predict(X_test)

        scores.append(model.score(X_test, Y_test))
    
    return (np.mean(scores), np.std(scores))

lm = LinearRegression()
mean_score, std = Cross_Validation(lm, X12, y12)
print("%0.2f accuracy with a standard deviation of %0.2f" % (mean_score, std))

lm = LinearRegression()
mean_score, std = Cross_Validation(lm, X1, y1)
print("%0.2f accuracy with a standard deviation of %0.2f" % (mean_score, std))

rf = RandomForestRegressor()
mean_score, std = Cross_Validation(rf, X12, y12)
print("%0.2f accuracy with a standard deviation of %0.2f" % (mean_score, std))
    




Tune the parameters of your model. 
Make predictions regarding the future values for some of your chosen variables. 

## Arima

# Installing all packages and importing files

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import warnings
import datetime as dt
import matplotlib.dates as mdates
warnings.filterwarnings('ignore')

In [ ]:
gen_1=pd.read_csv('./Plant_2_Generation_Data.csv')
gen_1.drop('PLANT_ID',1,inplace=True)

gen_2=pd.read_csv('./Plant_1_Generation_Data.csv')
gen_2.drop('PLANT_ID',1,inplace=True)

sens_1= pd.read_csv('./Plant_2_Weather_Sensor_Data.csv')
sens_1.drop('PLANT_ID',1,inplace=True)
#format datetime
gen_1['DATE_TIME']= pd.to_datetime(gen_1['DATE_TIME'],format='%Y-%m-%d %H:%M') #named plant 2 data as gen_1
gen_2['DATE_TIME']= pd.to_datetime(gen_2['DATE_TIME'],format='%d-%m-%Y %H:%M') #named plant 1 data as gen_2
sens_1['DATE_TIME']= pd.to_datetime(sens_1['DATE_TIME'],format='%Y-%m-%d %H:%M:%S')

# Daily Yield & AC-DC power
This plot shows that plant 1 is only outputting barely 10% of expected inverted output. Massive human error involved. Data cannot be used

In [ ]:
df_gen=gen_1.groupby('DATE_TIME').sum().reset_index() #group all items by the increments, which has several inverters. SUm all the inverter values into one. SO you get one value per time step for each collumn
df_gen['time']=df_gen['DATE_TIME'].dt.time #taking the time component from datetime and making a collumn

#Setting up plant 2 dataframe
df_gen2=gen_2.groupby('DATE_TIME').sum().reset_index()
df_gen2['time']=df_gen2['DATE_TIME'].dt.time

fig,ax = plt.subplots(ncols=2,nrows=1,dpi=100,figsize=(20,5))

# daily yield plot
#df_gen.plot(x='DATE_TIME',y='DAILY_YIELD',color='navy',ax=ax[0])


# AC & DC power plot
df_gen.set_index('time').drop('DATE_TIME',1)[['AC_POWER','DC_POWER']].plot(style='o',ax=ax[0]) #plant 2 plot
df_gen2.set_index('time').drop('DATE_TIME',1)[['AC_POWER','DC_POWER']].plot(style='o',ax=ax[1]) #plant 1 plot


#ax[0].set_title('Daily yield',)


ax[0].set_title('AC power & DC power during day hours for Plant 2')
ax[1].set_title('AC power & DC power during day hours for Plant 1')
ax[0].set_ylabel('kW',color='navy',fontsize=17)
plt.show()

pd.options.display.max_rows = 4000 #output text size
gen_1[3100:3120] #snapshot of plant 1 to illustrate this dataframe carries several inverters for each time step

# Illustrate that all inverter values have been added to total across each step

In [ ]:
df_gen[115:135] 

# Why original DAILY_YIELD and TOTAL_YIELD is wrong

Source Data mentions that DAILY_YIELD is a cumulative sum of power generated on that day, till that point in time. Power is a running rate, it cannot be Cumulative. Source Data also mentions TOTAL_YIELD is the total yield for the inverter till that point in time. However, cumulative
calculations are not consistent and therefore unusable. 

A snapshot of 3 days will be shown for both. DAILY_YIELD frequenlty dips, even though it has the correct general shape. This cannot be as it is reasonably a calculated value and is CUMULATIVE. The flat lines of TOTAL_YIELD also indicates to clue that it is not a proper cumulative,
especially since DAILY_YIELD is regularly producing output. There is also dips here, which are not possible.

In [ ]:
testDays = df_gen.loc[(df_gen['DATE_TIME'] > '2020-05-16') & (df_gen['DATE_TIME'] <= "2020-05-19")]


fig,ax= plt.subplots(ncols=2,dpi=100,figsize=(20,5))
testDays['DAILY_YIELD'].plot(ax=ax[0],color='navy') # DAILY_YIELD graph
testDays['TOTAL_YIELD'].plot(kind='bar',ax=ax[1],color='navy') #TOTAL_YIELD graph
fig.autofmt_xdate(rotation=45)
ax[0].set_title('Daily Yield')
ax[1].set_title('Total Yield')
ax[0].set_ylabel('kW',color='navy',fontsize=17)
plt.show()

#testDays

# Creating new and correct DAILY_YIELD and TOTAL_YIELD

In [ ]:
new_df=df_gen.copy()
new_df = new_df.drop(['DAILY_YIELD','TOTAL_YIELD'], 1) #Dropping DAILY_YIELD & TOTAL_YIELD. UNUSABLE

new_df["NEW_YIELD"] = (new_df['AC_POWER'] * 0.25) #Energy generation during this step

new_df['date']=new_df['DATE_TIME'].dt.date #taking date component of datetime into a new collumn


new_df["DAILY_YIELD"] = new_df.groupby('date').NEW_YIELD.cumsum() 
new_df["TOTAL_YIELD"] = new_df.NEW_YIELD.cumsum() 


#testDays = new_df.loc[(new_df['DATE_TIME'] > '2020-05-16') & (new_df['DATE_TIME'] <= "2020-05-19")]
#new_df = new_df.drop(['DAILY_YIELD','TOTAL_YIELD'], 1)


#df_gen=gen_1.groupby('DATE_TIME').sum().reset_index() #group all items by the increments, which has several inverters. SUm all the inverter values into one. SO you get one value per time step for each collumn
#df_gen['time']=df_gen['DATE_TIME'].dt.time #taking the time component from datetime and making a collumn

new_df[110:130] #snapshot of new dataframe (single step)


# Daily and Total Yield Plots
Replacing df_gen for easier carrydown

In [ ]:
df_gen = new_df #replacing df_gen for easier carrydown
daily_gen=df_gen.copy()
daily_gen['date']=daily_gen['DATE_TIME'].dt.date #taking date component of datetime into a new collumn

daily_gen=daily_gen.groupby('date').sum() #summing all the steps in a date so u get one value per date

fig,ax= plt.subplots(ncols=2,dpi=100,figsize=(20,5))
daily_gen['DAILY_YIELD'].plot(ax=ax[0],color='navy')
daily_gen['TOTAL_YIELD'].plot(kind='bar',ax=ax[1],color='navy')
fig.autofmt_xdate(rotation=45)
ax[0].set_title('Daily Yield')
ax[1].set_title('Total Yield')
ax[0].set_ylabel('kW',color='navy',fontsize=17)
plt.show()

daily_gen

# Task 2: Forecast
## Can we predict the power generation for next couple of days? 

Tune auto_arima function, a SEASONAL ARIMA(p,d,q) + (P,D,Q,m) model,on the last 4 days(384 observations) to see if our model can capture the last generation trend. 

In [ ]:
from pandas.tseries.offsets import DateOffset
! pip install pmdarima
from pmdarima.arima import auto_arima
from statsmodels.tsa.stattools import adfuller

New fixed gen_1 Data

In [ ]:
new_gen1=gen_1.copy()
new_gen1 = new_gen1.drop(['DAILY_YIELD','TOTAL_YIELD'], 1) #Dropping DAILY_YIELD & TOTAL_YIELD. UNUSABLE

new_gen1["NEW_YIELD"] = (new_gen1['AC_POWER'] * 0.25) #Energy generation during this step

new_gen1['date']=new_gen1['DATE_TIME'].dt.date #taking date component of datetime into a new collumn


new_gen1["DAILY_YIELD"] = new_gen1.groupby(['date', 'SOURCE_KEY']).NEW_YIELD.cumsum() 
new_gen1["TOTAL_YIELD"] = new_gen1.NEW_YIELD.cumsum() 

new_gen1 

#### Our data:

In [ ]:
pred_gen=new_gen1.copy()
pred_gen=pred_gen.groupby('DATE_TIME').sum()
pred_gen=pred_gen['DAILY_YIELD'][-288:].reset_index()
pred_gen.set_index('DATE_TIME',inplace=True)
pred_gen.head()

## Step 1: Testing for Stationarity


* A small p-value (typically ≤ 0.05) indicates strong evidence against the null hypothesis, so you reject the null hypothesis.

* A large p-value (> 0.05) indicates weak evidence against the null hypothesis, so you fail to reject the null hypothesis.

In [ ]:
result = adfuller(pred_gen['DAILY_YIELD'])
print('Augmented Dickey-Fuller Test:')
labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']

for value,label in zip(result,labels):
    print(label+' : '+str(value) )
    
if result[1] <= 0.05:
    print("strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root and is stationary")
else:
    print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

From the above, we can conclude that the data is non-stationary. Hence, we would need to use the “Integrated (I)” concept, denoted by value ‘d’ in time series to make the data stationary while building the Auto ARIMA model.

## Step 2: Split into train and test datasets to build the model on the training dataset and forecast using the test dataset.

In [ ]:
train=pred_gen[:192]
test=pred_gen[-96:]
plt.figure(figsize=(15,5))
plt.plot(train,label='Train',color='navy')
plt.plot(test,label='Test',color='darkorange')
plt.title('Last 4 days of daily yield',fontsize=17)
plt.legend()
plt.show()

train

## Step 3: Tune with the auto_arima function:



`P` is  The order of the seasonal component for the auto-regressive (AR) model.

`D` is The integration order of the seasonal process.

`Q` is The order of the seasonal component of the moving average (MA) model.
P and Q and be estimated similarly to p and q via auto_arima, and D can be estimated via a Canova-Hansen test, however m generally requires subject matter knowledge of the data.

Since we know that our observations are recorded at 15 minute intervals, (so for each day we have 96 observations) we can choose `m` parameter equal to 96 to capture daily trend.
To speed up the parameters search, I fixed a max order of 1 for P,D,Q paramaters in the seasonal component. 

In [ ]:
arima_model = auto_arima(train,
                         start_p=0,d=1,start_q=0,
                         max_p=4,max_d=4,max_q=4,
                         start_P=0,D=1,start_Q=0,
                         max_P=1,max_D=1,max_Q=1,m=96,
                         seasonal=True,
                         error_action='warn',trace=True,
                         supress_warning=True,stepwise=True,
                         random_state=20,n_fits=1)

## Step 4: Use the trained model which was built earlier to forecast daily yields

We use the trained model to forecast the last 96 observations of the test data, 17th June daily yield,  and then we will forecast daily yield for 18th and 19th June.

In [ ]:
future_dates = [test.index[-1] + DateOffset(minutes=x) for x in range(0,2910,15) ]

In [ ]:
prediction=pd.DataFrame(arima_model.predict(n_periods=96),index=test.index)
prediction.columns=['predicted_yield']

fig,ax= plt.subplots(ncols=2,nrows=1,dpi=100,figsize=(17,5))
ax[0].plot(train,label='Train',color='navy')
ax[0].plot(test,label='Test',color='darkorange')
ax[0].plot(prediction,label='Prediction',color='green')
ax[0].legend()
ax[0].set_title('Forecast on test set',size=17)
ax[0].set_ylabel('kW',color='navy',fontsize=17)


f_prediction=pd.DataFrame(arima_model.predict(n_periods=194),index=future_dates)
f_prediction.columns=['predicted_yield']
ax[1].plot(pred_gen,label='Original data',color='navy')
ax[1].plot(f_prediction,label='18th & 19th June',color='green')
ax[1].legend()
ax[1].set_title('Next days forecast',size=17)
plt.show()

## Model summary:

In [ ]:
arima_model.summary()

 Evaluate your model based on different metrics. 

 - R2, 
 - MSE,
 - RSME,
 - Cross validation

# References